In [9]:
import os
from typing import List
import numpy as np
import wfdb
from wfdb import processing
import random
import warnings
from tqdm import tqdm

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

DB_ALIASES = {
    "AFDB": "./../../mit-bih-atrial-fibrillation-database-1.0.0/files",
    "LTAFDB": "./../../long-term-af-database-1.0.0/files",
    "NSRDB": "./../../mit-bih-normal-sinus-rhythm-database-1.0.0/files",
}


def clean_annotation(annotation):
    sample, aux_note = annotation.sample, annotation.aux_note
    non_empty_indices = [i for i, note in enumerate(aux_note) if note != ""]
    clean_aux_note = [aux_note[i] for i in non_empty_indices]
    clean_sample = [sample[i] for i in non_empty_indices]
    return clean_sample, clean_aux_note


def get_ranges_afib(record_path: str, signal_len: int) -> List[List[int]]:
    annotation = wfdb.rdann(record_path, "atr")
    sample, aux_note = clean_annotation(annotation)
    ranges_interest = []

    for i, label in enumerate(aux_note):
        if label == "(AFIB":
            afib_start = sample[i]
            afib_end = signal_len if i == len(sample) - 1 else sample[i + 1] - 1
            ranges_interest.append([afib_start, afib_end])

    return ranges_interest


def get_record_ids(db_alias: str) -> List[str]:
    with open(f"{DB_ALIASES[db_alias]}/RECORDS") as f:
        record_ids = [line.strip() for line in f.readlines()]

    if db_alias == "AFDB":
        record_ids.remove("00735")
        record_ids.remove("03665")

    return record_ids


def extract_segments(db_alias):
    record_ids = get_record_ids(db_alias)

    segment_size = 5

    record_ids_size = len(record_ids)

    for record_index, record_id in enumerate(record_ids):
        record_path = os.path.join(DB_ALIASES[db_alias], record_id)
        _, ecg_metadata = wfdb.rdsamp(record_path)
        signal_len = ecg_metadata["sig_len"]

        extract_intervals = (
            get_ranges_afib(record_path, signal_len)
            if db_alias in ["AFDB", "LTAFDB"]
            else [[0, signal_len - 1]]
        )

        rec = wfdb.rdrecord(record_name=record_path)

        stack_rr = np.empty((0, 5), dtype=int)
        stack_recording = []

        print(f"{record_index + 1}/{record_ids_size}")

        for start_index, end_index in tqdm(extract_intervals):
            ann = wfdb.rdann(
                record_path,
                sampfrom=start_index,
                sampto=end_index,
                extension="atr" if db_alias == "NSRDB" else "qrs",
            )

            ann_ms = (ann.sample / ann.fs) * 1000
            rr_interval = processing.calc_rr(ann_ms, fs=ann.fs)
            num_segments = (len(ann.sample) - 1) // segment_size

            if num_segments <= 0:
                continue

            last_segment = num_segments * segment_size

            for i in range(0, last_segment, segment_size):
                start_index = ann.sample[i]
                end_index = ann.sample[i + segment_size]

                rr_segment = rr_interval[i : i + segment_size]
                stack_rr = np.vstack((stack_rr, rr_segment))

                rec_seg = rec.p_signal[start_index:end_index]
                stack_recording.append(rec_seg)

            print(stack_recording)

            break

        break

        np.save(
            file=f"./{db_alias}/{record_index}_{record_id}_rri_segment", arr=stack_rr
        )

        stack_recording = np.array(stack_recording, dtype=object)
        np.save(
            file=f"./{db_alias}/{record_index}_{record_id}_recording_segment",
            arr=stack_recording,
        )

In [10]:
folder_path = "./AFDB"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

extract_segments("AFDB")

1/23


  0%|          | 0/7 [00:00<?, ?it/s]

[array([[-1.33 , -0.18 ],
       [-1.155, -0.195],
       [-1.04 , -0.135],
       ...,
       [-0.865, -1.46 ],
       [-0.98 , -1.74 ],
       [-0.875, -1.9  ]]), array([[-0.755, -2.06 ],
       [-0.505, -2.145],
       [-0.19 , -2.055],
       ...,
       [-1.29 , -1.535],
       [-1.63 , -1.875],
       [-1.57 , -2.13 ]]), array([[-1.38 , -2.335],
       [-1.065, -2.39 ],
       [-0.745, -2.29 ],
       [-0.48 , -1.965],
       [-0.355, -1.54 ],
       [-0.27 , -1.155],
       [-0.225, -0.835],
       [-0.21 , -0.585],
       [-0.215, -0.465],
       [-0.205, -0.335],
       [-0.205, -0.27 ],
       [-0.205, -0.225],
       [-0.18 , -0.21 ],
       [-0.18 , -0.205],
       [-0.15 , -0.21 ],
       [-0.14 , -0.205],
       [-0.175, -0.18 ],
       [-0.175, -0.16 ],
       [-0.19 , -0.14 ],
       [-0.16 , -0.16 ],
       [-0.19 , -0.175],
       [-0.16 , -0.16 ],
       [-0.06 , -0.165],
       [ 0.005, -0.16 ],
       [-0.025, -0.185],
       [-0.05 , -0.19 ],
       [-0.075, -0.15

In [2]:
folder_path = "./NSRDB"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

extract_segments("NSRDB")

1/18


100%|██████████| 1/1 [00:01<00:00,  1.78s/it]


2/18


100%|██████████| 1/1 [00:02<00:00,  2.01s/it]


3/18


...

100%|██████████| 1/1 [00:01<00:00,  1.43s/it]


17/18


100%|██████████| 1/1 [00:01<00:00,  1.60s/it]


18/18


100%|██████████| 1/1 [00:02<00:00,  2.44s/it]


In [3]:
folder_path = "./LTAFDB"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

extract_segments("LTAFDB")

1/84


100%|██████████| 44/44 [00:31<00:00,  1.39it/s]


2/84


100%|██████████| 53/53 [00:10<00:00,  5.25it/s]


3/84


...

100%|██████████| 29/29 [00:28<00:00,  1.01it/s]


83/84


100%|██████████| 1044/1044 [15:56<00:00,  1.09it/s]


84/84


100%|██████████| 1/1 [00:01<00:00,  1.42s/it]


In [ ]:
base_pos = []
base_pos.append(extract_segments("AFDB", segment_size=5, limit_seg=100))
base_pos.append(extract_segments("LTAFDB", segment_size=5, limit_seg=100))
base_pos = [item for sublist in base_pos for item in sublist]
random_samples_pos = random.sample(base_pos, 200)

base_neg = []
base_neg.append(extract_segments("NSRDB", segment_size=5, limit_seg=200))
base_neg = [item for sublist in base_neg for item in sublist]
random_samples_neg = random.sample(base_neg, 200)